Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

df = pd.read_csv('ml-latest-small/ratings.csv')

df_sort = df.groupby('userId').timestamp.agg(['min', 'max', 'count'])
df_sort = df_sort[df_sort['count'] > 100]
df_sort['time_life'] = df_sort['max'] - df_sort['min']
df_sort.time_life.mean()

40080507.4496124

Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [2]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [3]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [4]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [5]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [6]:
table_revenue = rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer').set_index('client_id')
table_revenue = table_revenue.fillna(0)
table_revenue['sum_revenue'] = table_revenue.sum(axis=1)
table_result = table_revenue.merge(client_base, on='client_id', how='left')

In [7]:
table_revenue

,rzd_revenue,auto_revenue,air_revenue,sum_revenue
client_id,,,,
111,1093.0,0.0,0.0,1093.0
112,2810.0,0.0,0.0,2810.0
113,10283.0,57483.0,0.0,67766.0
114,5774.0,83.0,0.0,5857.0
115,981.0,912.0,81.0,1974.0
116,0.0,4834.0,4.0,4838.0
117,0.0,98.0,13.0,111.0
118,0.0,0.0,173.0,173.0


In [8]:
table_result

,client_id,rzd_revenue,auto_revenue,air_revenue,sum_revenue,address
0,111,1093.0,0.0,0.0,1093.0,Комсомольская 4
1,112,2810.0,0.0,0.0,2810.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,67766.0,Левобережная 1а
3,114,5774.0,83.0,0.0,5857.0,Мира 14
4,115,981.0,912.0,81.0,1974.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,4838.0,Строителей 18
6,117,0.0,98.0,13.0,111.0,Панфиловская 33
7,118,0.0,0.0,173.0,173.0,Мастеркова 4


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Геоданные - это специфический тип данных, который, в контексте "визитов и покупок" используется очень редко, только при геоаналитики. Поэтому оптимально было бы хранить их отдельной таблице с следующими данными user_id и время визита/покупки. И подключал бы их к таблице только при необходимости. Также было бы неплохо хранить в этой таблице данные о часовом поясе, которые можно получить с помощью интернет-сервисов. Информация о реальном времени у пользователя даст нам лучшее представление о времени суток, когда визит/покупка были осуществлены(так как в нашей базе сохраняется локальное время визита/покупки)

2. Разные координаты при смене часового пояса, дадут нам информацию о реальном времени у пользователя, когда он совершил визит/покупку.

3. Геоаналитика может дать нам очень много информации о том, как распределены пользователи по городам. Из неё мы можем:
- более точно спланировать рекламу товаров;
- время у пользоватей, когда эту рекламу лучше демонстрировать;
- организовать правильно складскую, логистические службы и розничную сеть, если мы занимаемся торговлей;
- по характеру геоданных мы можем понять(изменяются или нет в рамках одной сессии), какие пользователи больше пользуются мобильной версией, а какие с стационарных компьютеров).